In [2]:
import os
import shutil
import sys
import time
from PIL import Image
from datetime import date, datetime, timedelta

sys.path.insert(0, '..')
from utils import mongodb
from pesagensbalanca import cursor_pesagensbalanca, extract_to

## Generate vazios database

#### Necessário acesso ao Servidor ajna.labin.rf08.srf (e habilitação no AJNA) ou acesso ao Servidor mongo.labin.rf08.srf (e senha do banco)

In [4]:
#if os.path.exists('pesagensbalanca_cropped'):
#    shutil.rmtree('pesagensbalanca_cropped')
#    os.remove('pesos.txt')

In [3]:
start = datetime(2017, 7, 1)
end = datetime.today()
limit = 500
s0 = time.time()
cursor = cursor_pesagensbalanca(mongodb, start, end, limit, crop=True)
count = extract_to(cursor, crop=True)
s1 = time.time()
print('{:0.2f} segundos para processar {:d} registros'.format((s1 - s0), count))


KeyError: 'pesagens'

## Example: Training a very basic convolutional network

In [2]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [3]:
model = models.Sequential([
  layers.Conv2D(16, (5, 5), activation='relu',
                padding='same',
                input_shape=(144, 288, 3)),
  layers.MaxPooling2D(pool_size=(2, 4)),
  layers.Dropout(0.25),
  layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(128, (3, 3), activation='relu'),
  layers.Conv2D(128, (3, 3), activation='relu'),
  layers.Conv2D(128, (1, 1), activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.Conv2D(512, (1, 1), activation='relu'),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(1)
 
])

model.compile(optimizer=optimizers.Adam(lr=0.001), loss='mean_squared_error', metrics=['mse'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 144, 288, 16)      1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 72, 72, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 72, 72, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        1

In [ ]:
df = pd.read_csv('pesos.txt')
df.head()

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df, directory='pesagensbalanca_cropped',
    x_col='id', y_col='peso', has_ext=False,
    class_mode="other", target_size=(144, 288),
    batch_size=128,
    subset='training'
)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df, directory='pesagensbalanca_cropped',
    x_col='id', y_col='peso', has_ext=False,
    class_mode="other", target_size=(144, 288),
    batch_size=128,
    subset='validation'
)


Found 756 images belonging to 2 classes.
Found 188 images belonging to 2 classes.


In [5]:

model.fit(train_generator, epochs=50,
         validation_data = validation_generator,
         validation_steps = validation_generator.samples // 128)

Train for 6 steps, validate for 3 steps
Epoch 1/50
6/6 [==============================] - 16s 3s/step - loss: 0.6244 - acc: 0.6508 - val_loss: 0.6163 - val_acc: 0.6833
Epoch 2/50
6/6 [==============================] - 10s 2s/step - loss: 0.5399 - acc: 0.7434 - val_loss: 0.5776 - val_acc: 0.8444
Epoch 3/50
6/6 [==============================] - 10s 2s/step - loss: 0.4429 - acc: 0.8267 - val_loss: 0.5215 - val_acc: 0.8278
Epoch 4/50
6/6 [==============================] - 11s 2s/step - loss: 0.4264 - acc: 0.8016 - val_loss: 0.5061 - val_acc: 0.7944
Epoch 5/50
6/6 [==============================] - 10s 2s/step - loss: 0.3415 - acc: 0.8558 - val_loss: 0.4364 - val_acc: 0.8833
Epoch 6/50
6/6 [==============================] - 10s 2s/step - loss: 0.2478 - acc: 0.8981 - val_loss: 0.4808 - val_acc: 0.7889
Epoch 7/50
6/6 [==============================] - 10s 2s/step - loss: 0.2989 - acc: 0.9074 - val_loss: 0.3537 - val_acc: 0.8444
Epoch 8/50
6/6 [==============================] - 10s 2s/step - 

In [6]:
model.save_weights('gerabasespesagens_cropped.hdf5')

In [7]:
#Fine tunning
for layer in model.layers:
    layer.trainable = True

for layer in model.layers[:-7]:
    layer.trainable = False

reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.1, patience=4,
                          verbose=1, min_delta=1e-2, mode='min')

model.compile(optimizer=optimizers.Adam(lr=0.000001), loss='mean_squared_error', metrics=['mse'])
for l in model.layers:
    print(l.name, l.trainable)

conv2d False
max_pooling2d False
dropout False
conv2d_1 False
max_pooling2d_1 False
dropout_1 False
conv2d_2 False
max_pooling2d_2 False
dropout_2 False
conv2d_3 False
conv2d_4 False
conv2d_5 False
max_pooling2d_3 False
dropout_3 False
conv2d_6 True
conv2d_7 True
conv2d_8 True
flatten True
dense True
dropout_4 True
dense_1 True


In [8]:
model.fit_generator(train_generator, epochs=20,
                    callbacks=[reduce_lr],
                   validation_data = validation_generator,
                   validation_steps = validation_generator.samples // 128)

Epoch 1/20
6/6 [==============================] - 10s 2s/step - loss: 0.0885 - acc: 0.9749 - val_loss: 0.1888 - val_acc: 0.9333
Epoch 2/20
6/6 [==============================] - 8s 1s/step - loss: 0.0949 - acc: 0.9630 - val_loss: 0.1966 - val_acc: 0.9333
Epoch 3/20
6/6 [==============================] - 8s 1s/step - loss: 0.0902 - acc: 0.9722 - val_loss: 0.1939 - val_acc: 0.9278
Epoch 4/20
6/6 [==============================] - 7s 1s/step - loss: 0.0944 - acc: 0.9735 - val_loss: 0.1951 - val_acc: 0.9278
Epoch 5/20
5/6 [========================>.....] - ETA: 0s - loss: 0.0952 - acc: 0.9781
Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.
6/6 [==============================] - 8s 1s/step - loss: 0.0909 - acc: 0.9802 - val_loss: 0.1961 - val_acc: 0.9389
Epoch 6/20
6/6 [==============================] - 8s 1s/step - loss: 0.0911 - acc: 0.9669 - val_loss: 0.2009 - val_acc: 0.9278
Epoch 7/20
6/6 [==============================] - 8s 1s/step - loss: 0.1100 - acc

In [11]:
model.save_weights('gerabasespesagens_cropped_tunned.hdf5')